In [ ]:
# Идеология задачи (рекомендательной системы, которая будет предлагать 35 наиболее реальных участий)
# Если рассчитывается 35, то выдавать меньше - просто снижать вероятность успеха 
# Из пула нужна выбирать наиболее вероятные через введение внутреннего скора (логику определяем сами)

In [1]:
import pandas as pd
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [2]:
#считывание данных 
train_data = pd.read_csv(r"C:\Users\Mr Alex\Downloads\data\train_data.csv", sep=';')
train_labels = pd.read_csv(r"C:\Users\Mr Alex\Downloads\data\train_labels.csv", sep=';')
test_data = pd.read_csv(r"C:\Users\Mr Alex\Downloads\data\test_data.csv", sep=';')

In [3]:
#Заполнение пропусков строкой 'None'
train_data = train_data.fillna('None')
test_data = test_data.fillna('None')

#### Обработка okpd2 и additional_code

In [4]:
train_data['okpd2_or_additional_code'] = train_data[['okpd2_code',
                                                     'additional_code']].apply(lambda x: x[0] if x[1] == 'None' else x[1], axis=1)
test_data['okpd2_or_additional_code'] = test_data[['okpd2_code', 'additional_code']].apply(
    lambda x: x[0] if x[1] == 'None' else x[1], axis=1)

In [5]:
#Просмотр частоты значений 
train_data['okpd2_or_additional_code'].value_counts()

drug    65646
32.5    36292
71.1    17836
20.5    15591
26.2    14783
        ...  
80.3        1
1.7         1
14.2        1
2.3         1
9.9         1
Name: okpd2_or_additional_code, Length: 657, dtype: int64

#### Формирование описания тендера

In [6]:
train_data['text_description_tender'] = train_data['purchase_name'] + " " + train_data['lot_name'] + " " + train_data['okpd2_names'] + " " + train_data['additional_code_names'] + " " + train_data['item_descriptions']

In [8]:
train_data.head(3).T

,0,1,2
pn_lot_anon,pn_lot_7031618,pn_lot_7808247,pn_lot_7009496
fz,44fz,44fz,44fz
region_code,2,2,2
min_publish_date,2019-08-26,2019-03-12,2019-03-14
purchase_name,Услуги по проведению финансового аудита,Приобретение призов для проведения конкурса по...,приобретение призов для проведения мероприятия...
lot_name,None,None,None
lot_price,123500,144000,124200
okpd2_code,69.2,26.4,26.2
okpd2_names,Услуги по проведению финансового аудита,"Телефоны головные, наушники и комбинированные ...",Устройства запоминающие внешние
additional_code,None,None,None


#### Работа с историей участий поставщика

In [9]:
inn_kpp_history = pd.merge(train_labels, train_data[['pn_lot_anon','region_code', 'okpd2_or_additional_code']], on=['pn_lot_anon'])

In [10]:
inn_kpp_history.head()

,pn_lot_anon,participant_inn_kpp_anon,is_winner,fz,region_code,okpd2_or_additional_code
0,pn_lot_1770702,inn_kpp_3661899,1,44fz,1,42.1
1,pn_lot_1058704,inn_kpp_3661899,1,44fz,1,42.9
2,pn_lot_4186044,inn_kpp_3661899,1,44fz,1,42.1
3,pn_lot_6882961,inn_kpp_3661899,1,44fz,1,42.9
4,pn_lot_5821609,inn_kpp_3661899,1,44fz,1,42.1


In [11]:
#группировка по поставщику 
inn_kpp_history = inn_kpp_history.groupby('participant_inn_kpp_anon').apply(lambda x: [
    list(x['pn_lot_anon']),
    list(x['is_winner']), 
    list(x['fz']), 
    list(x['region_code']), 
    list(x['okpd2_or_additional_code'])]).apply(pd.Series)

In [12]:
inn_kpp_history = inn_kpp_history.reset_index()
inn_kpp_history.columns = ['participant_inn_kpp_anon', 'list_pn_lot_anon',
                           'list_is_winner', 'list_fz', 'list_region_code',
                           'list_okpd2_or_additional_code']

In [13]:
inn_kpp_history.head()

,participant_inn_kpp_anon,list_pn_lot_anon,list_is_winner,list_fz,list_region_code,list_okpd2_or_additional_code
0,inn_kpp_1000018,"[pn_lot_5330449, pn_lot_648665, pn_lot_81155, ...","[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, ...","[44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44f...","[64, 50, 50, 77, 50, 64, 77, 77, 77, 77, 50, 7...","[58.2, 26.2, 26.4, 26.3, 26.2, 58.2, 26.2, 27...."
1,inn_kpp_1001268,"[pn_lot_7608031, pn_lot_9976372, pn_lot_671651...","[1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, ...","[44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44f...","[77, 79, 79, 79, 79, 27, 79, 79, 79, 79, 79, 7...","[86.9, 86.1, 86.2, 86.9, 86.9, 86.9, 86.9, 86...."
2,inn_kpp_1002518,"[pn_lot_5287076, pn_lot_7474816, pn_lot_121000...","[0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, ...","[44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44f...","[41, 27, 82, 65, 65, 65, 41, 41, 41, 41, 82, 4...","[30.3, 27.1, 27.1, 25.1, 30.9, 29.2, 25.9, 25...."
3,inn_kpp_1003261,[pn_lot_2647783],[1],[223fz],[54],[28.2]
4,inn_kpp_1003291,"[pn_lot_7794721, pn_lot_4767800, pn_lot_915979...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44f...","[50, 12, 50, 50, 51, 69, 11, 12, 11, 12, 12, 1...","[32.5, 32.5, 32.5, 32.5, 32.5, 32.5, 32.5, 21...."


In [20]:
comb['region_code'] = train_data['region_code'].astype('object')

In [23]:
comb = [train_data, test_data]

In [24]:
for c in comb:
    c['region_code'] = c['region_code'].astype('object')

In [29]:
train_data.describe(include='all')

,pn_lot_anon,fz,region_code,min_publish_date,purchase_name,lot_name,lot_price,okpd2_code,okpd2_names,additional_code,additional_code_names,item_descriptions,okpd2_or_additional_code,text_description_tender
count,778901,778901,778901.0,778901,778901,778901,7.789010e+05,778901,778901,778901,778901,778901,778901,778901
unique,778901,2,92.0,475,528202,100723,NaN,349,43430,644,12194,449290,657,713526
top,pn_lot_2388707,44fz,77.0,2020-03-30,Поставка канцелярских товаров,None,NaN,None,None,None,None,None,drug,Бензин автомобильный АИ-92 экологического клас...
freq,1,649744,82252.0,5518,3839,649668,NaN,91694,91694,558068,558068,83734,65646,304
mean,NaN,NaN,NaN,NaN,NaN,NaN,3.420793e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,1.008930e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,9.900000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,3.054280e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,9.914156e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
train_data['fz'].value_counts(normalize=True)

44fz     0.83418
223fz    0.16582
Name: fz, dtype: float64

In [25]:
train_data['list_fz'].unique()

KeyError: 'list_fz'

#### Рекомендательная система

Выбирается 35 случайный процедур из подвыборки с совпадением региона и ОКПД2 кода актуальной процедуры с регионами и ОКПД2 кодами из истории участия поставщика

In [14]:
inn_kpp_recommendation = []
similarity_score = 1
for inn_kpp in tqdm_notebook(inn_kpp_history.values):
    participant_inn_kpp_anon, list_participant_inn_kpp_anon, list_is_winner, list_fz, list_region_code, list_okpd2_or_additional_code = inn_kpp
    # подвыборка с совпадением региона и ОКПД2 кода актуальной с историей поставщика
    recommendation = test_data[test_data['region_code'].isin(
        list_region_code) & test_data['okpd2_or_additional_code'].isin(list_okpd2_or_additional_code)]
    if recommendation.shape[0] >= 35:
        # выбор 35 случайных актуальных процедур из подвыборки
        recommendation = recommendation.sample(35)['pn_lot_anon'].values
        for actual_pn_lot in recommendation:
            inn_kpp_recommendation.append(
                [participant_inn_kpp_anon, actual_pn_lot, similarity_score])

#### Пример формирования файла рекомендаций

In [ ]:
recommendation = pd.DataFrame(inn_kpp_recommendation, columns=['inn_kpp', 'actual_recommended_pn_lot', 'similarity_score'])
recommendation.to_csv("TushkanProphets.csv", index=False, sep=';')

In [ ]:
recommendation

# Подсчёт метрик

In [ ]:
test_labels = pd.read_csv("ваша тестирующая выборка", sep=';')

In [ ]:
true = set((test_labels['pn_lot_anon'] + "_" + test_labels['participant_inn_kpp_anon']).values)

In [ ]:
pred = set((recommendation['actual_recommended_pn_lot'] + "_" + recommendation['inn_kpp']).values)

In [ ]:
intersection = len(true.intersection(pred))

In [ ]:
print(f"Точность: {intersection / len(pred) * 100}")
print(f"Полнота: {intersection / len(true) * 100}")